In [1]:
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
import pandas as pd
import time

In [ ]:
from pyecharts.globals import CurrentConfig

CurrentConfig.ONLINE_HOST = "https://cdn.kesci.com/lib/pyecharts_assets/"

In [ ]:
现状
累计确诊

In [ ]:
df_global_confirm = pd.read_csv('/home/kesci/input/covid192815/time_series_covid19_confirmed_global.csv')

def replace_country_name(val):
    replace_dic = {
        'US':'United States',
        'Congo (Brazzaville)':'Congo',
        'Congo (Kinshasa)':'Dem. Rep. Congo',
        'South Sudan':'S. Sudan',
        'Central African Republic':'Central African Rep.',
        'Western Sahara':'W. Sahara',
        'Laos':'Lao PDR',
        'Korea, South':'Korea'
    }
    for k, v in replace_dic.items():
        try:
            val = val.replace(k,v)
        except AttributeError:
            pass
    return val
    
df_global_confirm.iloc[:,1] = df_global_confirm.iloc[:,1].apply(replace_country_name)

df_global_country_confirm = df_global_confirm.groupby(['Country/Region']).sum().reset_index()

In [ ]:
geo = Geo()
data_pair, data_pair_high = [], []
for idx, row in df_global_confirm.iloc[:,[0,1,2,3,-1]].iterrows():
    if int(row[-1]) > 5e6:
        label = row['Country/Region'] if row['Province/State'] is np.nan  else row['Province/State']
        geo.add_coordinate(label, row['Long'], row['Lat'])
        data_pair_high.append([label, int(row[-1])])
        
for idx, row in df_global_country_confirm.iloc[:,[0,-1]].iterrows():
    data_pair.append([row['Country/Region'], int(row[-1])])

In [ ]:

# 用于geo和bar中的标签显示
fmt_js = """function (params) {return params.name+':'+Number(params.value[2]);}"""

"""
Bar:
展示每天top10国家的数据
放在grid的左下角
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#1773c3',
                                         'shadowColor': '#1773c3',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
# 标签格式设置
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# 全局配置项
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=4e7),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", pos_top='55%', pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=5e6,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(219,112,147,0.4)', 'rgba(238,25,27,0.9)', 'rgba(238,25,27,1)']))
# 转换xy轴
bar.reversal_axis()

"""
map图：
在整个grid中，map是被用来当作底图使用的
地图的颜色都在map中来配置
"""
mp = Map()
mp.add(
    "累计确诊人数",
    data_pair,
    "world",
    zoom=1.2,
    is_map_symbol_show=False,
    is_roam=False)
# 关闭map的标签显示 & 地图风格设置
mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#091632',
                       'borderColor': '#1773c3',
                       'shadowColor': '#1773c3',
                       'shadowBlur': 20,
                       'opacity': 0.6
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="累计确诊病例", 
                              subtitle='数据更新日期：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global_confirm.columns[-1], '%m/%d/%y'))),
                              pos_bottom='8%', pos_left='75%',
                              subtitle_textstyle_opts=opts.TextStyleOpts(font_size=12),
                              title_textstyle_opts=opts.TextStyleOpts(font_size=25,font_weight='bold')),
    legend_opts=opts.LegendOpts(is_show=False),
)

"""
GEO图：
geo在整个grid中是用于展示几个主要国家的数据
地图的样式都是由map控制
"""
geo = Geo()

# 这里将geo的地图透明度配置为0
geo.add_schema(
    maptype="world",
    is_roam=False,
    zoom=1.2,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", data_pair_high, type_='effectScatter', symbol_size=JsCode("""function (val) {return val[2] / 4e5;}"""))
# 显示标签配置
geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='white',
        font_size=12,
        font_weight='bold',
        formatter=JsCode(fmt_js))
        )

# 将三个图组合起来，配置好各自的位置
grid = (
    Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

In [ ]:

累计死亡

In [ ]:
df_global_death = pd.read_csv('/home/kesci/input/covid192815/time_series_covid19_deaths_global.csv')

def replace_country_name(val):
    replace_dic = {
        'US':'United States',
        'Congo (Brazzaville)':'Congo',
        'Congo (Kinshasa)':'Dem. Rep. Congo',
        'South Sudan':'S. Sudan',
        'Central African Republic':'Central African Rep.',
        'Western Sahara':'W. Sahara',
        'Laos':'Lao PDR',
        'Korea, South':'Korea'
    }
    for k, v in replace_dic.items():
        try:
            val = val.replace(k,v)
        except AttributeError:
            pass
    return val
    
df_global_death.iloc[:,1] = df_global_death.iloc[:,1].apply(replace_country_name)

df_global_country_death = df_global_death.groupby(['Country/Region']).sum().reset_index()

In [ ]:
geo = Geo()
data_pair, data_pair_high = [], []
for idx, row in df_global_death.iloc[:,[0,1,2,3,-1]].iterrows():
    if int(row[-1]) > 1e5:
        label = row['Country/Region'] if row['Province/State'] is np.nan  else row['Province/State']
        geo.add_coordinate(label, row['Long'], row['Lat'])
        data_pair_high.append([label, int(row[-1])])
        
for idx, row in df_global_country_death.iloc[:,[0,-1]].iterrows():
    data_pair.append([row['Country/Region'], int(row[-1])])

In [ ]:

# 用于geo和bar中的标签显示
fmt_js = """function (params) {return params.name+':'+Number(params.value[2]);}"""

"""
Bar:
展示每天top10国家的数据
放在grid的左下角
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#1773c3',
                                         'shadowColor': '#1773c3',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
# 标签格式设置
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# 全局配置项
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=1e6),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", pos_top='55%', pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=2e5,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(219,112,147,0.4)', 'rgba(238,25,27,0.9)', 'rgba(238,25,27,1)']))
# 转换xy轴
bar.reversal_axis()

"""
map图：
在整个grid中，map是被用来当作底图使用的
地图的颜色都在map中来配置
"""
mp = Map()
mp.add(
    "累计死亡病例",
    data_pair,
    "world",
    zoom=1.2,
    is_map_symbol_show=False,
    is_roam=False)
# 关闭map的标签显示 & 地图风格设置
mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#091632',
                       'borderColor': '#1773c3',
                       'shadowColor': '#1773c3',
                       'shadowBlur': 20,
                       'opacity': 0.6
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="累计死亡病例", 
                              subtitle='数据更新日期：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global_death.columns[-1], '%m/%d/%y'))),
                              pos_bottom='8%', pos_left='75%',
                              subtitle_textstyle_opts=opts.TextStyleOpts(font_size=12),
                              title_textstyle_opts=opts.TextStyleOpts(font_size=25,font_weight='bold')),
    legend_opts=opts.LegendOpts(is_show=False),
)

"""
GEO图：
geo在整个grid中是用于展示几个主要国家的数据
地图的样式都是由map控制
"""
geo = Geo()

# 这里将geo的地图透明度配置为0
geo.add_schema(
    maptype="world",
    is_roam=False,
    zoom=1.2,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", data_pair_high, type_='effectScatter', symbol_size=JsCode("""function (val) {return val[2] / 1e4;}"""))
# 显示标签配置
geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='white',
        font_size=12,
        font_weight='bold',
        formatter=JsCode(fmt_js))
        )

# 将三个图组合起来，配置好各自的位置
grid = (
    Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

In [ ]:
累计治愈

In [ ]:
df_global_recover = pd.read_csv('/home/kesci/input/covid192815/time_series_covid19_recovered_global.csv')

def replace_country_name(val):
    replace_dic = {
        'US':'United States',
        'Congo (Brazzaville)':'Congo',
        'Congo (Kinshasa)':'Dem. Rep. Congo',
        'South Sudan':'S. Sudan',
        'Central African Republic':'Central African Rep.',
        'Western Sahara':'W. Sahara',
        'Laos':'Lao PDR',
        'Korea, South':'Korea'
    }
    for k, v in replace_dic.items():
        try:
            val = val.replace(k,v)
        except AttributeError:
            pass
    return val
    
df_global_recover.iloc[:,1] = df_global_recover.iloc[:,1].apply(replace_country_name)

df_global_country_recover = df_global_recover.groupby(['Country/Region']).sum().reset_index()

In [ ]:
geo = Geo()
data_pair, data_pair_high = [], []
for idx, row in df_global_recover.iloc[:,[0,1,2,3,-1]].iterrows():
    if int(row[-1]) > 2e6:
        label = row['Country/Region'] if row['Province/State'] is np.nan  else row['Province/State']
        geo.add_coordinate(label, row['Long'], row['Lat'])
        data_pair_high.append([label, int(row[-1])])
        
for idx, row in df_global_country_recover.iloc[:,[0,-1]].iterrows():
    data_pair.append([row['Country/Region'], int(row[-1])])

In [ ]:

# 用于geo和bar中的标签显示
fmt_js = """function (params) {return params.name+':'+Number(params.value[2]);}"""

"""
Bar:
展示每天top10国家的数据
放在grid的左下角
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#1773c3',
                                         'shadowColor': '#1773c3',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
# 标签格式设置
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# 全局配置项
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=3e7),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", 
                                              pos_top='55%', 
                                              pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=6e5,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['#ffffff', 'rgba(0,128,0,1)']))
# 转换xy轴
bar.reversal_axis()

"""
map图：
在整个grid中，map是被用来当作底图使用的
地图的颜色都在map中来配置
"""
mp = Map()
mp.add(
    "累计治愈病例",
    data_pair,
    "world",
    zoom=1.2,
    is_map_symbol_show=False,
    is_roam=False)
# 关闭map的标签显示 & 地图风格设置
mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#ffffff',
                       'borderColor': '#1773c3',
                       'shadowColor': '#1773c3',
                       'shadowOffsetY': 1,
                       'shadowOffsetX': 1,
                       'shadowBlur': 0,
                       'opacity': 1
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="累计治愈病例", 
                              subtitle='数据更新日期：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global_recover.columns[-1], '%m/%d/%y'))),
                              pos_bottom='8%', 
                              pos_left='75%',
                              subtitle_textstyle_opts=opts.TextStyleOpts(font_size=12),
                              title_textstyle_opts=opts.TextStyleOpts(font_size=25,
                                                                      font_weight='bold')),
    legend_opts=opts.LegendOpts(is_show=False),
)

"""
GEO图：
geo在整个grid中是用于展示几个主要国家的数据
地图的样式都是由map控制
"""
geo = Geo()

# 这里将geo的地图透明度配置为0
geo.add_schema(
    maptype="world",
    is_roam=False,
    zoom=1.2,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", 
        data_pair_high, 
        type_='scatter', 
        symbol_size=1)
# 显示标签配置
geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='#F5F5F5',
        font_size=13,
        font_weight='bold',
        formatter=JsCode(fmt_js))
        )

# 将三个图组合起来，配置好各自的位置
grid = (
    Grid(init_opts=opts.InitOpts(theme='dark', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

In [ ]:
现状

In [ ]:
# 把确诊，死亡，治愈三个时间序列数据拼接起来
df_global_country = pd.merge(df_global_country_confirm.iloc[:,[0,-2,-1]], 
                             df_global_country_death.iloc[:,[0,-1]], 
                             on='Country/Region')
df_global_country = pd.merge(df_global_country, 
                             df_global_country_recover.iloc[:,[0,-1]], 
                             on='Country/Region')
df_global_country.columns = ['Country/Region', 'confirm', 'confirm_add', 'death', 'recover']

# 数据分别转为list，并将日期格式化为‘yyyy-MM-dd’格式
data_x, data_y = [],[]
for idx, row in df_global_country.iterrows():
    if int(row['confirm']) > 10000:
        data_x.append(round(int(row['death']) / int(row['confirm']),4))
        data_y.append([
            round(int(row['recover']) / int(row['confirm']),4),
            int(row['death']), 
            int(row['confirm_add']), 
            int(row['confirm']),
            row['Country/Region'],
            int(row['recover']),
            int(row['confirm'])-int(row['death']) - int(row['recover']),
                      ]
                     )



In [ ]:
tool_js = """

function (obj) {
            var value = obj.value;
            var schema = [{name: 'death_rate', index: 0, text: '死亡率'},
                          {name: 'recover_rate', index: 1, text: '治愈率'},
                          {name: 'death', index: 2, text: '死亡病例'},
                          {name: 'confirm_add', index: 3, text: '新增病例'},
                          {name: 'confirm', index: 4, text: '确诊病例'},
                          {name: 'Country/Region', index: 5, text: '国家/地区'},
                          {name: 'recover', index: 6, text: '治愈病例'},
                          {name: 'current_confirm', index: 7, text: '当前确诊'}];
            return '<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">'
                + value[5] + ' 现状：'
                + '</div>'
                + schema[0].text + '：' + value[0]*100+'%' +  '<br>'
                + schema[1].text + '：' + value[1]*100+'%' +  '<br>'
                + schema[7].text + '：' + value[7] + '<br>'
                + schema[3].text + '：' + value[3] + '<br>'
                + schema[4].text + '：' + value[4] + '<br>'
                + schema[2].text + '：' + value[2] + '<br>'
                + schema[6].text + '：' + value[6] + '<br>';
        }
"""

itemStyle = {
    'shadowBlur': 3,
    'shadowOffsetX': 5,
    'shadowOffsetY': 5,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    "borderColor": 'rgb(220,220,220)',
    "borderWidth": 3
}



visualMap = [
        {
            'left': 'right',
            'top': '12%',
            'dimension': 4,
            'min': 100,
            'max': 5e5,
            'itemWidth': 30,
            'itemHeight': 120,
            'calculable': True,
            'precision': 0.1,
            'text': ['圆形大小：确诊病例'],
            'textGap': 30,
            'textStyle': {
                'color': '#fff'
            },
            'inRange': {
                'symbolSize': [5, 60]
            },
            'controller': {
                'inRange': {
                    'color': ['#c23531']
                },
                'outOfRange': {
                    'color': ['#444']
                }
            }
        },
        {
            'left': 'right',
            'top': '40%',
            'dimension': 2,
            'min': 100,
            'max': 5e4,
            'itemWidth': 30,
            'itemHeight': 120,
            'calculable': True,
            'precision': 0.1,
            'text': ['颜色：死亡病例'],
            'textGap': 30,
            'textStyle': {
                'color': '#fff'
            },
            'inRange': {
                'color': ['rgb(34,139,34)', 'rgb(255,0,0)']
            },
            'controller': {
                'inRange': {
                    'color': ['rgb(34,139,34)', 'rgb(255,0,0)'],
                    'colorLightness':0.6
                },
                'outOfRange': {
                    'color': ['#444']
                }
            }
        },
        {
            'left': 'right',
            'bottom': '5%',
            'dimension': 7,
            'calculable': True,
            'min': 1000,
            'max': 5e4,
            'itemHeight': 120,
            'precision': 0.1,
            'text': ['明暗：当前确诊'],
            'textGap': 30,
            'textStyle': {
                'color': '#fff'
            },
            'inRange': {
                'colorLightness': [1, 0.5]
            },
            'controller': {
                'inRange': {
                    'color': ['#c23531']
                },
                'outOfRange': {
                    'color': ['#444']
                }
            }
        }
    ]

scatter = (Scatter(init_opts=opts.InitOpts(theme='chalk',width='980px', height='800px'))
            .add_xaxis(data_x)
            .add_yaxis("", 
                       data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle)
            .set_global_opts(yaxis_opts=opts.AxisOpts(name='治愈率', 
                                                      type_="value", 
                                                      is_scale=True, 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(
                                                          linestyle_opts=opts.LineStyleOpts(color='#eee'))),
                             xaxis_opts=opts.AxisOpts(name='死亡率', 
                                                      type_="value", 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(
                                                          linestyle_opts=opts.LineStyleOpts(color='#eee'))),
                             tooltip_opts=opts.TooltipOpts(is_show=True, 
                                                           background_color='#222', 
                                                           border_color='#777', 
                                                           border_width=1, 
                                                           formatter=JsCode(tool_js)),
                             legend_opts=opts.LegendOpts(is_show=True, 
                                                         pos_right=10,
                                                         textstyle_opts=opts.TextStyleOpts(
                                                             color='#fff',
                                                             font_size=16)),
                             title_opts=opts.TitleOpts(title="国家/地区现状", 
                                  subtitle='数据更新日期：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global_recover.columns[-1], '%m/%d/%y'))),
                                  pos_top='1%', pos_left='center',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=24)),
                             visualmap_opts=visualMap)
            )




scatter.render_notebook()

In [ ]:

趋势
时间序列

In [ ]:
# 把确诊，死亡，治愈三个时间序列数据拼接起来
df_global = pd.merge(df_global_confirm.iloc[:,4:].sum().reset_index(), 
                     df_global_death.iloc[:,4:].sum().reset_index(), 
                     on='index')
df_global = pd.merge(df_global, 
                     df_global_recover.iloc[:,4:].sum().reset_index(), 
                     on='index')
df_global.columns = ['date', 'confirm', 'death', 'recover']

# 数据分别转为list，并将日期格式化为‘yyyy-MM-dd’格式
data_x, data_y_confirm, data_y_death, data_y_recover = [],[],[],[]
for idx, row in df_global.iterrows():
    label = time.strftime("%Y-%m-%d",time.strptime(row['date'], '%m/%d/%y'))
    data_x.append(label)
    data_y_confirm.append(int(row['confirm']))
    data_y_death.append(int(row['death']))
    data_y_recover.append(int(row['recover']))


In [ ]:
# 用于图表中文字显示
msg = "['截止{}:', '当前确诊：{:,}例', '当前死亡：{:,}例','当前治愈：{:,}例'].join('\\n\\n')"

msg = msg.format(time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[-1,0], '%m/%d/%y')),
                 df_global.iloc[-1, 1],
                 df_global.iloc[-1, 2],
                 df_global.iloc[-1, 3])


In [ ]:
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(139,0,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}


line = Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='980px', height='600px'))
line.add_xaxis(data_x)
line.add_yaxis("累计确诊", 
        data_y_confirm, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("累计死亡", 
        data_y_death, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("累计治愈", 
        data_y_recover, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="全球疫情趋势",
                                               subtitle='数据更新日期：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[-1,0], '%m/%d/%y'))),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                       font_size=20)),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='5%', 
                                                 pos_left="70%", 
                                                 selected_mode='multiple',
                                                 legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                   font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, 
                                                                              color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, 
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value / 10000 +'万';}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, 
                                                                                  color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, 
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            )
                                        ),
                     graphic_opts=[opts.GraphicGroup(
                                        graphic_item=opts.GraphicItem(
                                            bounding="raw",
                                            left=250,
                                            top=150,
                                            z=100,
                                        ),
                                        children=[
                                            opts.GraphicRect(
                                                graphic_item=opts.GraphicItem(
                                                    left="center", top="center", z=100
                                                ),
                                                graphic_shape_opts=opts.GraphicShapeOpts(
                                                    width=200, height=100
                                                ),
                                                graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                    fill="rgba(0,0,0,0.3)"
                                                ),
                                            ),
                                            opts.GraphicText(
                                                graphic_item=opts.GraphicItem(
                                                    left="center", top="center", z=100
                                                ),
                                                graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                                                    text=JsCode(msg),
                                                    font="bold 12px Microsoft YaHei",
                                                    graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                        fill="#fff"
                                                    ),
                                                ),
                                            ),
                                        ],
                                    )
                                ],
                    )      
line.render_notebook()